In [6]:
import csv

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF
from collections import defaultdict
from bs4 import BeautifulSoup, NavigableString
from soupselect import select

from lxml.html.soupparser import fromstring
from bs4 import UnicodeDammit

import lxml.etree as ET


def uri_to_file_name(uri):
    return uri.replace("/", "-")

sessions = {}

xpath_abstract = '''//div[@class='panel-body']/div[1]/text()'''

with open("data/sc.csv", "r") as sessions_file:
    for row in csv.DictReader(sessions_file, ['title', 'link', 'speaker']):  
        session_id = (row['title'])
        filename = "data/sessions/" + uri_to_file_name(row['link']) + '.html'
        page = open(filename).read()
        soup = fromstring(page)
        ab = soup.xpath(xpath_abstract)
        if len(ab) >0 :
            abstract = ab[0].replace('\n', ' ').replace('\r', '').replace('\r\n', '').encode('utf-8','ignore')
            title = row['title']
            sessions[row['link']] = {'title':title , 'abstract':abstract}
           

In [ ]:
            
        
corpus = []
titles = []
for id, session in sorted(sessions.iteritems(), key=lambda t: t[0]):
    corpus.append(session["abstract"] + session["title"])
    titles.append(session["title"])

n_topics = 15
n_top_words = 50
n_features = 6000

vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1,1), min_df = 0, stop_words = 'english')
# vectorizer = CountVectorizer(analyzer='word', ngram_range=(1,1), min_df = 0, stop_words = 'english')

In [19]:
matrix =  vectorizer.fit_transform(corpus)
feature_names = vectorizer.get_feature_names()
feature_names

[u'000',
 u'10',
 u'16846',
 u'1976',
 u'2010',
 u'2015',
 u'3d',
 u'6th',
 u'76',
 u'88',
 u'able',
 u'abstract',
 u'accelerometer',
 u'accessing',
 u'accounts',
 u'accuracy',
 u'accurate',
 u'achieve',
 u'actimeter',
 u'activities',
 u'actually',
 u'ad',
 u'addresses',
 u'adopted',
 u'advantages',
 u'advent',
 u'advertisement',
 u'advertising',
 u'advice',
 u'aggregate',
 u'aid',
 u'aims',
 u'algorithms',
 u'allows',
 u'analyses',
 u'analysis',
 u'analysts',
 u'analytic',
 u'analytics',
 u'analyzed',
 u'angus',
 u'animated',
 u'animation',
 u'annotated',
 u'annotation',
 u'anomalies',
 u'ansible',
 u'anticheating',
 u'apac',
 u'apache',
 u'api',
 u'apis',
 u'app',
 u'application',
 u'applications',
 u'applied',
 u'apply',
 u'appreciate',
 u'approach',
 u'appropriate',
 u'apps',
 u'architecture',
 u'area',
 u'art',
 u'artificial',
 u'asertthat',
 u'asia',
 u'assembly',
 u'attaching',
 u'attempts',
 u'attracted',
 u'attributes',
 u'audience',
 u'author',
 u'autodoc',
 u'automated',
 u'

In [11]:
import lda
import numpy as np

vocab = feature_names

model = lda.LDA(n_topics=4, n_iter=500, random_state=1)
model.fit(matrix)
topic_word = model.topic_word_
n_top_words = 5

for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-n_top_words:-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

doc_topic = model.doc_topic_
for i in range(0, len(titles)):
    print("{} (top topic: {})".format(titles[i], doc_topic[i].argmax()))
    print(doc_topic[i].argsort()[::-1][:3])

IndexError: index 0 is out of bounds for axis 0 with size 0